In [28]:
import json
import random
import re
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import spatial

import pandas as pd
import numpy as np

import os
import datetime
import string
import nltk

In [29]:
# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

wordnet_lemmatizer = WordNetLemmatizer()
stopwords = set(nltk.corpus.stopwords.words('english'))
print(len(stopwords))
print(stopwords)

def tokenize_lemmatize_remove_stopwords(text):
    tokens = word_tokenize(text)
    tokens = [wordnet_lemmatizer.lemmatize(word) for word in tokens]
    # tokens = [word for word in tokens if word not in stopwords and not bool(re.search(r'\d', word))]
    tokens = [word for word in tokens if not bool(re.search(r'\d', word))]
    return tokens

def clean_job_description(df: pd.DataFrame):
    df['job_description_cleaned'] = df['Job Description'].str.lower() \
        .str.replace(r"\n+", " ", regex=True) \
        .str.replace(r'[^\w\s]', ' ', regex=True) \
        .apply(tokenize_lemmatize_remove_stopwords).str.join(' ') \
        .str.replace(r'\bsr\b', 'senior', regex=True) \
        .str.replace(r'\bjr\b', 'junior', regex=True)

def clean_job_title(df: pd.DataFrame):
    df['Job Title clean'] = df['Job Title'].str.lower() \
        .str.replace(r"\(.*?\)", "", regex=True) \
        .str.replace(r"\n+", " ", regex=True) \
        .str.replace('#NAME?', '-') \
        .str.replace('with', '-') \
        .str.replace(r'[^\w\s]', ' ', regex=True) \
        .apply(tokenize_lemmatize_remove_stopwords).str.join(' ') \
        .str.replace(r'\bsr\b', 'senior', regex=True) \
        .str.replace(r'\bjr\b', 'junior', regex=True) \

    
# def get_word_counts_df(df):
#     temp = df["job_description_cleaned"].str.split(expand=True).stack().value_counts()
#     ret = pd.DataFrame(temp, columns=["count"])
#     return ret

179
{'during', "weren't", 'hasn', 'no', "haven't", "wouldn't", 'were', 'is', 'than', 'in', 'same', 'itself', "couldn't", 'she', 'y', "didn't", 'just', 'being', 'some', 'my', 'had', 'those', 'yours', 'against', 'himself', 'them', 'out', "aren't", 'whom', 'your', "shan't", "you'd", 'that', 'be', "wasn't", 'their', "hasn't", 'after', 'will', 'or', 'his', 'won', 'are', 'below', 's', 'more', 'where', 'have', 'then', 'me', "won't", 'only', 'not', 't', 'we', 'if', 'by', 'about', 'here', 'any', 'too', 'through', 'most', 'at', 'why', 'needn', 'and', 'wouldn', 'haven', "doesn't", "you'll", 'few', 'themselves', 'theirs', 'there', 'm', 'ourselves', 'her', 'didn', 'herself', 'into', "it's", 'doing', 'once', "don't", 'does', 'doesn', 'a', "needn't", 'who', 'as', 'how', 'of', 'll', 'to', 'isn', 'this', 'now', "should've", "that'll", 'shouldn', 'did', 'between', 'so', 'am', 'couldn', "you're", 'can', 'wasn', 'with', 'on', 're', 'which', 'it', "she's", 'such', 'him', 'ours', 'been', 'hadn', 'our', 'ove

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alexw\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alexw\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alexw\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [30]:
# df = pd.read_csv('data_jobs_data/data/DataScientist.csv').drop(['Unnamed: 0', 'index'],axis=1)
# # df = pd.read_csv('scraper results/data scientist 900.csv')
# df

In [31]:
def remove_duplicates(df:pd.DataFrame, exclude=None):
    subset_columns = ['Job Title', 'Company Name', 'Location', 'Salary Estimate', 'Rating', 'Job Description']
    if exclude:
        if isinstance(exclude, list):
            subset_columns = [a for a in subset_columns if a not in exclude]
        else:
            subset_columns.remove(exclude)
    
    df.sort_values(subset_columns, inplace=True)

    df['Duplicate Count'] = df.groupby(subset_columns)['Job Title'].transform('count')
    print(df['Duplicate Count'].value_counts())
    df.drop_duplicates(subset=subset_columns, inplace=True, keep='last')
    df.reset_index(drop=True,inplace=True)
    

In [32]:
['Job Title', 'Company Name', 'Location', 'Salary Estimate', 'Rating', 'Job Description', 'Founded', 'Industry', 'Revenue', 'Sector', 'Size', 'Type', 'Easy Apply']

['Job Title',
 'Company Name',
 'Location',
 'Salary Estimate',
 'Rating',
 'Job Description',
 'Founded',
 'Industry',
 'Revenue',
 'Sector',
 'Size',
 'Type',
 'Easy Apply']

In [33]:
state_abbreviations = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA',
    'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO',
    'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
    'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH',
    'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT',
    'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}
state_abbreviations_vals = set(state_abbreviations.values())

# Function to convert full state names to abbreviations
def convert_state_name(state):
    return state_abbreviations.get(state, state)


In [34]:

# Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply
def convert_sqft_to_num(x,agg):
    if x == '-1' or not x or x == np.NaN:
        return None
    tokens = x.split('-')

    try:
        if len(tokens) == 2:
            if agg == 'mean':
                return (float(tokens[0])+float(tokens[1]))/2
            elif agg == 'min':
                return min(float(tokens[0]),float(tokens[1]))
            elif agg == 'max':
                return max(float(tokens[0]),float(tokens[1]))
        return float(x)
    except Exception as e:
        print(tokens)
        print(e)
        return None

def isrevenue(x):
    if x=="Non-Applicable":
        return 0
    elif x=="(USD)":
        return 1

def salary_type(s):
    if not isinstance(s,str):
        return None
    if 'Per Hour' in s:
        return 'Per Hour'
    else:
        return 'Annual (K)'


def reorder_dataframe_columns_inplace(df:pd.DataFrame, desired_order, prefix='delete_'):
    filtered_order = [col for col in desired_order if col in df.columns]
    df.rename(columns={col: prefix + col for col in filtered_order}, inplace=True)
    for col in filtered_order:
        df[col] = df[prefix + col]
        del df[prefix + col]

def preprocessing(df: pd.DataFrame):
    df.rename(columns = {'Type of ownership':'Type'}, inplace = True) # ,'Salary Estimate':'salary'
    for col in {'Unnamed: 0', 'index', 'Competitors', 'Headquarters'}:
        if col in df.columns:
            df.drop(col, axis=1, inplace=True)

    reorder_dataframe_columns_inplace(df, ['Job Title', 'Company Name', 'Location', 'Salary Estimate', 'Rating', 'Job Description', 'Founded', 
                                           'Industry', 'Revenue', 'Sector', 'Size', 'Type', 'Easy Apply', 'Duplicate Count'])

    df['Company Name'] = df['Company Name'].apply(lambda x:str(x).strip('\n'))
    df['Company Name'] = df['Company Name'].apply(lambda x:str(x).split('\n')[0] if '\n' in x else x)

    if df['Easy Apply'].dtype != bool:
        df['Easy Apply'] = df['Easy Apply'].replace(['-1'],0).replace(['FALSE'],1)
        df['Easy Apply'] = df['Easy Apply'].astype(bool)


    # df.dropna(how='any',axis=0,inplace=True)
    df.fillna(-1,inplace=True)
    df.reset_index(drop=True,inplace=True)

    df['Founded']=df['Founded'].astype(int)

    df['Size'] = df['Size'].apply(lambda x:x.replace('to','-'))
    df['Size'] = df['Size'].apply(lambda x:x.replace('+',' - 0'))
    df['Size'] = df['Size'].apply(lambda x:x.replace('employees',' '))
    df['Size'] = df['Size'].apply(lambda x:x.replace('Employees',' ')).str.replace(' ','')

    df['Salary Type'] = df['Salary Estimate'].apply(salary_type)


    # # Drop the rows where salary missing
    # df.drop(df[df['Salary Estimate'].isna() | (df['Salary Estimate'] == '-1')].index, inplace=True)

    # df['Salary Estimate'].replace([-1, None], '-1')
    df['Salary Estimate'] = df['Salary Estimate'].str.replace('[^0-9\-.]', '', regex=True).str.strip('.')
    df['Salary Estimate'].fillna('-1',inplace=True)

    df['min_salary'] = df['Salary Estimate'].apply(lambda x: convert_sqft_to_num(x, 'min'))
    df['max_salary'] = df['Salary Estimate'].apply(lambda x: convert_sqft_to_num(x, 'max'))

    # replace -1 with nan
    df['Revenue'] = df['Revenue'].replace(['-1', 'Unknown / Non-Applicable'],None)
    df['Sector'] = df['Sector'].replace(['-1'],None)
    df['Type'] = df['Type'].replace(['-1', 'Unknown'],None)
    df['Size'] = df['Size'].replace(['-1', 'Unknown'],None)
    df['Founded'] = df['Founded'].replace([-1],None)
    df['Rating'] = df['Rating'].replace([-1],None)
    
    
    df['State'] = df['Location'].apply(lambda x: str(x).split(', ')[1] if ', ' in str(x) else x)\
        .apply(lambda x: state_abbreviations.get(x, x)) #\
#            .apply(lambda x: x if x in state_abbreviations_vals else None)
    
    
def extract_years_experience(description):
    # Updated regex pattern to match numbers with an optional '+' sign followed by 'year' or 'years'
    # This will match '2 years', '5-7 years', '3+ years', '3 year', etc.
    pattern = r'(\d+\+?|\d+-\d+)\s+years?'

    # Search for the pattern in the description
    match = re.search(pattern, description.lower())
    if match:
        return match.group(1)
    else:
        return None




def clean_data(df):
    print(df.shape)
    # remove_duplicates(df, exclude='Location') # remove location duplicates of the same role
    remove_duplicates(df)
    print(df.shape)
    preprocessing(df)
    print(df.shape)
    
    clean_job_description(df)
    clean_job_title(df)

    # Apply the function to create a new column
    df['Years Experience'] = df['Job Description'].apply(extract_years_experience)

    
    return df

In [35]:
# pd.set_option('display.max_colwidth', 50)
# df_new = pd.read_csv('combined scraper results/data engineer 2465.csv')
# clean_data(df_new)
# df_new.to_csv('combined scraper results/data engineer 2465 clean.csv',index=False)
# df_new.head(4)

In [36]:
path = 'combined scraper results'
for fname in os.listdir(path):
    if 'clean' in fname:
        continue
    print(fname)
    fn = fname.strip('.csv')
    data = pd.read_csv(os.path.join(path, fn+'.csv'))
    clean_data(data)
    data.to_csv(os.path.join(path, fn+' clean.csv'),index=False)
    # print(data.head(4))
    print(data.shape)

business analyst.csv


(10830, 14)
Duplicate Count
1.0    10791
Name: count, dtype: int64
(10830, 14)
(10830, 18)
(10830, 21)
data analyst.csv
(9923, 14)
Duplicate Count
1.0    9882
Name: count, dtype: int64
(9923, 14)
(9923, 18)
(9923, 21)
data engineer.csv
(2465, 14)
Duplicate Count
1.0    2450
Name: count, dtype: int64
(2465, 14)
(2465, 18)
(2465, 21)
data scientist.csv
(5539, 14)
Duplicate Count
1.0    5514
Name: count, dtype: int64
(5539, 14)
(5539, 18)
(5539, 21)


In [37]:
path = os.path.join('data_jobs_data','data')
for fname in os.listdir(path):
    if 'clean' in fname:
        continue
    print(fname)
    fn = fname.strip('.csv')
    data = pd.read_csv(os.path.join(path, fn+'.csv'))
    clean_data(data)
    data.to_csv(os.path.join(path, fn+' clean.csv'),index=False)
    # print(data.head(4))
    print(data.shape)

BusinessAnalyst.csv
(4092, 15)
Duplicate Count
1    4092
Name: count, dtype: int64
(4092, 16)
(4092, 18)
(4092, 21)
DataAnalyst.csv
(5631, 15)
Duplicate Count
1.0    5629
Name: count, dtype: int64
(5631, 16)
(5631, 18)
(5631, 21)
DataEngineer.csv
(2528, 15)
Duplicate Count
1    2504
2      24
Name: count, dtype: int64
(2516, 16)
(2516, 18)
(2516, 21)
DataScientist.csv
(3909, 17)
Duplicate Count
1    3909
Name: count, dtype: int64
(3909, 18)
(3909, 18)
(3909, 21)
